In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# Data Transforms
> Fastrl transforms for iterating through environments

In [ ]:

if __name__ == '__main__':
    import os
    os.environ['MKL_THREADING_LAYER']="GNU"
#     os.environ['MKL_SERVICE_FORCE_INTEL']="1"

    
    import torch.multiprocessing as mp
    try:mp.set_start_method('spawn',force=True)
    except Exception as e: print(e)
    
    import torch
        # Python native modules
    import os
    # Third party libs
    from fastai.torch_basics import default_device
#     from fastai.data.all import DataLoader
    from torch.utils.data import DataLoader
    # Local modules
    from fastrl.data.block import *

#     import numpy as np
#     import gym

    dl=DataLoader(TestDataset(DQN(),device=default_device()),num_workers=1,batch_size=50)
    for x in dl:
        print(x)

tensor([[ 0.0029,  0.2243, -0.0268, -0.2859],
        [-0.0092,  0.2149,  0.0272, -0.3320],
        [-0.0032,  0.1805,  0.0036, -0.2889],
        [ 0.0207,  0.2242,  0.0251, -0.3285],
        [-0.0018,  0.1739, -0.0041, -0.3238],
        [-0.0148,  0.2259,  0.0138, -0.3320],
        [-0.0209,  0.2315,  0.0425, -0.2785],
        [ 0.0137,  0.2178, -0.0129, -0.2591],
        [-0.0478,  0.1588,  0.0134, -0.2671],
        [-0.0123,  0.1720,  0.0453, -0.3096],
        [ 0.0170,  0.1961,  0.0188, -0.2525],
        [-0.0008,  0.1811, -0.0423, -0.2739],
        [-0.0218,  0.2282, -0.0361, -0.2851],
        [ 0.0457,  0.2301, -0.0402, -0.2755],
        [-0.0208,  0.1965, -0.0096, -0.2739],
        [ 0.0109,  0.2173, -0.0476, -0.3193],
        [ 0.0323,  0.2370, -0.0153, -0.2499],
        [ 0.0178,  0.2359, -0.0393, -0.2687],
        [ 0.0159,  0.1767,  0.0286, -0.2339],
        [-0.0192,  0.1692, -0.0251, -0.2674],
        [-0.0031,  0.2286, -0.0155, -0.2981],
        [ 0.0347,  0.1735, -0.0456

In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
No notebooks were modified
